In [1]:
%matplotlib inline
%load_ext autoreload
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import pathlib
import time
from datetime import datetime
import scipy.optimize as opt
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)
plt.style.use("ggplot")
mpl.pyplot.rcParams['figure.figsize'] = (5.0, 4.0)
plt.rcParams["font.size"] = 13
#mpl.rcParams["font.family"] = 'Osaka'

import sys
sys.path.append('../code/')

from simulation_hash_price import *
from simulation_fixed_path import make_stats
from hash import *
from plot_path import *

## 1. Compute the possible drift range

* Split the dataset from Oct 2018 to Dec 2018 into 10 periods with equal length.
  - each period contains approx. 220 data points
* Then estimate the empirical drift(/hour) in each period
* the estimated drift range is $[-0.00183, 0.00063]$.

In [2]:
path = '../data/return_and_volatility.csv'
df = pd.read_csv(path)
df = df.dropna()

In [3]:
df.head()

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume,[S(t+1) - S(t)]/S(t),deviation^2
1,1.538360e+12,43374.04167,BTCUSD,6621.30,6639.05,6607.91,6609.99,59.028481,-0.001708,2.193410e-06
2,1.538360e+12,43374.08333,BTCUSD,6609.99,6620.30,6609.75,6613.17,32.324636,0.000481,5.015410e-07
3,1.538360e+12,43374.12500,BTCUSD,6613.17,6630.00,6611.12,6625.35,32.193978,0.001842,4.280280e-06
4,1.538370e+12,43374.16667,BTCUSD,6625.35,6629.96,6596.40,6603.02,53.909761,-0.003370,9.880220e-06
5,1.538370e+12,43374.20833,BTCUSD,6603.02,6603.02,6592.64,6599.55,41.811981,-0.000526,8.904950e-08


In [5]:
df.shape

(2207, 10)

In [6]:
# split df into N=10 periods
nbins = 10
period_list = []
for i in range(nbins):
    period_list.append(i)

In [7]:
s_qcut = pd.qcut(df['Date'], nbins, labels=period_list)
df['period'] = s_qcut
df.head()

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume,[S(t+1) - S(t)]/S(t),deviation^2,period
1,1.538360e+12,43374.04167,BTCUSD,6621.30,6639.05,6607.91,6609.99,59.028481,-0.001708,2.193410e-06,0
2,1.538360e+12,43374.08333,BTCUSD,6609.99,6620.30,6609.75,6613.17,32.324636,0.000481,5.015410e-07,0
3,1.538360e+12,43374.12500,BTCUSD,6613.17,6630.00,6611.12,6625.35,32.193978,0.001842,4.280280e-06,0
4,1.538370e+12,43374.16667,BTCUSD,6625.35,6629.96,6596.40,6603.02,53.909761,-0.003370,9.880220e-06,0
5,1.538370e+12,43374.20833,BTCUSD,6603.02,6603.02,6592.64,6599.55,41.811981,-0.000526,8.904950e-08,0


In [9]:
def empirical_estimate(df):
    diff = df['[S(t+1) - S(t)]/S(t)'].values
    mu = np.mean(diff)
    sigma = np.sqrt(np.var(diff, ddof=1))
    return mu, sigma

In [10]:
empirical_estimate(df)

(-0.0002271055617127322, 0.008657880544230548)

In [11]:
mu, sigma = empirical_estimate(df)
sigma/np.sqrt(60)

0.0011177275720419373

In [12]:
mu_list = []
sigma_list = []
for i in range(nbins):
    df_temp = df[df['period']==i]
    mu, sigma = empirical_estimate(df_temp)
    mu_list.append(mu)
    sigma_list.append(sigma)

In [14]:
mu_list = np.array(mu_list)
mu_list.min(), mu_list.max()

(-0.00183907665, 0.000631057355158371)

## 2. Simulation with variable drift

* Assume a positive relation between hash rate and drift
  - more hash rate -> less probability of system's collapse -> the present value of bitcoin goes up
* Let $\mu(t)$ be period-$t$ drift and assume the following version of geometric Brownian motion:
$$
S(t+1) - S(t) = \mu(t) S(t) B(t) + \sigma S(t) \sqrt{B(t)} \epsilon (t),
$$
where $\epsilon(t)$ i.i.d.-drawn from $\mathcal{N}(0,1)$.
* We assume the following functional form:
$$
\mu(t) = \mu_{min} + \frac{\mu_{max}-\mu_{min}}{H_{max} - H_{min}}(H(t) - H_{min}).
$$

In [15]:
# Rescale the unit of drift to per minute:
mu_list_min = mu_list/60
mu_list_min.min(), mu_list_min.max()

(-3.06512775e-05, 1.051762258597285e-05)

In [20]:
(-3.06512775e-05, 1.051762258597285e-05)

(-3.06512775e-05, 1.051762258597285e-05)

In [13]:
# generate epsilons
num_iter = 5000

df_epsilon = pd.DataFrame()
for iter in range(num_iter):
    df_epsilon['iter_{}'.format(iter)] = np.random.normal(size=20000)
df_epsilon.to_csv('/Volumes/Data/research/BDA/simulation/sim_epsilons_{}obs.csv'.format(num_iter), index=False)    

/var/folders/st/whqq9b792k79l_xbsbn054pm0000gn/T/ipykernel_12056/3923294646.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_epsilon['iter_{}'.format(iter)] = np.random.normal(size=20000)


In [14]:
df_epsilon = pd.read_csv('/Volumes/Data/research/BDA/simulation/sim_epsilons_{}obs.csv'.format(num_iter))

In [15]:
df_epsilon.shape

(20000, 5000)

In [2]:
path = '../data/BTCdata_presim.csv'
df = pd.read_csv(path)
df['time'] = pd.to_datetime(df['time'])
df = df.rename(columns={'blocktime': 'block_times', 'price': 'prices', 'probability of success /Eh': 'winning_rates'})

In [17]:
# test with small number of iter
generate_simulation_data_asert(num_iter=2, prev_data=df)

In [19]:
print(datetime.now())

2022-02-05 20:58:41.180275


In [7]:
# takes ?? hours
print(datetime.now())
generate_simulation_data(num_iter=5000, prev_data=df)
print(datetime.now())
generate_simulation_data_DAA0(num_iter=5000, prev_data=df)
print(datetime.now())
generate_simulation_data_asert(num_iter=5000, prev_data=df)
print(datetime.now())

2022-02-06 23:42:34.958555
2022-02-07 03:11:15.430810
2022-02-07 05:08:10.639728
2022-02-07 07:02:05.218612


## 3. Compute statistics

In [2]:
dir_sim = '/Volumes/Data/research/BDA/simulation/'
filelist = [dir_sim+'hash-price_DAA_asert_blocktime_ps0_5000obs_T=None.csv',
            dir_sim+'hash-price_DAA-0_blocktime_ps0_5000obs.csv',
            dir_sim+'hash-price_DAA-1_blocktime_ps0_5000obs_T=None.csv',
            dir_sim+'hash-price_DAA-2_blocktime_ps0_5000obs_T=None.csv']
df_stats = make_stats(filelist=filelist, dir_sim=dir_sim)

In [3]:
df_stats

,hash-price_DAA_asert_blocktime_ps0_5000obs_T=None.csv,hash-price_DAA-0_blocktime_ps0_5000obs.csv,hash-price_DAA-1_blocktime_ps0_5000obs_T=None.csv,hash-price_DAA-2_blocktime_ps0_5000obs_T=None.csv
mean,10.150479,9.817022,30.197126,10.057853
std,10.224103,9.820488,72.783264,10.256323
over60,1.000000,1.000000,1.000000,1.000000
over120,0.122600,0.055800,0.916800,0.188600
over180,0.000400,0.000000,0.855000,0.001400
